# Multi model

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import os
import csv
from docx import Document
import json
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,accuracy_score, precision_recall_curve,f1_score, confusion_matrix, RocCurveDisplay
from sklearn.metrics import classification_report

## Reading training data

In [41]:
merged_df=pd.read_csv('Merge_data.csv')

In [42]:

merged_df['departure.scheduledTime'] = pd.to_datetime(merged_df['departure.scheduledTime'])


merged_df['day_of_week'] = merged_df['departure.scheduledTime'].dt.day_name()  # e.g., Monday, Tuesday
merged_df['hour_of_day'] = merged_df['departure.scheduledTime'].dt.hour         # e.g., 14, 8
merged_df['month_of_year'] = merged_df['departure.scheduledTime'].dt.month_name()  # e.g., December


In [43]:

df_encoded = pd.get_dummies(merged_df, columns=['status','type','month_of_year','day_of_week','hour_of_day','airline.name',
       'airline.iataCode', 'airline.icaoCode', 'flight.number',
       'flight.iataNumber', 'flight.icaoNumber',])
df_encoded

,departure.iataCode,departure.icaoCode,departure.terminal,departure.scheduledTime,departure.estimatedTime,departure.actualTime,departure.estimatedRunway,departure.actualRunway,arrival.iataCode,arrival.icaoCode,...,flight.icaoNumber_uae603,flight.icaoNumber_uae605,flight.icaoNumber_uae607,flight.icaoNumber_uae623,flight.icaoNumber_uae625,flight.icaoNumber_uae8619,flight.icaoNumber_uae8621,flight.icaoNumber_voz6098,flight.icaoNumber_voz6173,flight.icaoNumber_vsv850
0,lhe,opla,m,2023-07-20 20:50:00,2023-07-20t20:00:00.000,2023-07-20 20:15:00,2023-07-20t20:15:00.000,2023-07-20t20:15:00.000,ruh,oerk,...,0,0,0,0,0,0,0,0,0,0
1,lhe,opla,NaN,2023-07-18 15:05:00,2023-07-18t15:05:00.000,NaN,NaN,NaN,khi,opkc,...,0,0,0,0,0,0,0,0,0,0
2,lhe,opla,NaN,2023-07-23 09:50:00,NaN,NaN,NaN,NaN,khi,opkc,...,0,0,0,0,0,0,0,0,0,0
3,lhe,opla,m,2023-07-26 23:30:00,2023-07-26t23:30:00.000,2023-07-26 23:51:00,2023-07-26t23:51:00.000,2023-07-26t23:51:00.000,kwi,okkk,...,0,0,0,0,0,0,0,0,0,0
4,lhe,opla,m,2023-07-20 11:35:00,2023-07-20t17:15:00.000,NaN,NaN,NaN,dxb,omdb,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12109,khi,opkc,m,2023-08-02 18:40:00,2023-08-02t18:44:00.000,2023-08-02 18:54:00,2023-08-02t18:54:00.000,2023-08-02t18:54:00.000,auh,omaa,...,0,0,0,0,0,0,0,0,0,0
12110,khi,opkc,m,2023-08-10 04:00:00,2023-08-10t04:00:00.000,2023-08-10 03:58:00,2023-08-10t03:58:00.000,2023-08-10t03:58:00.000,doh,othh,...,0,0,0,0,0,0,0,0,0,0
12111,khi,opkc,m,2023-08-01 16:35:00,2023-08-01t16:35:00.000,2023-08-01 17:10:00,2023-08-01t17:10:00.000,2023-08-01t17:10:00.000,dxb,omdb,...,0,0,0,0,0,0,0,0,0,0
12112,khi,opkc,m,2023-08-12 07:00:00,NaN,NaN,NaN,NaN,isb,opis,...,0,0,0,0,0,0,0,0,0,0


## Normalizing

In [44]:
from sklearn.preprocessing import MinMaxScaler
columns_to_normalize=['type_departure', 'month_of_year_April', 'month_of_year_August',
       'month_of_year_December', 'month_of_year_February',
       'month_of_year_January', 'month_of_year_July', 'month_of_year_June',
       'month_of_year_March', 'month_of_year_May', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'status_cancelled','status_active','Temp Max','Temp Avg', 'Temp Min', 'Dew Point Max', 'Dew Point Avg',
       'Dew Point Min', 'Humidity Max', 'Humidity Avg', 'Humidity Min',
       'Wind Speed Max', 'Wind Speed Avg', 'Wind Speed Min', 'Pressure Max',
       'Pressure Avg', 'Pressure Min']
scaler = MinMaxScaler()


df_encoded[columns_to_normalize] = scaler.fit_transform(df_encoded[columns_to_normalize])

In [45]:
from sklearn.preprocessing import StandardScaler
for index, i in enumerate(merged_df['delay_in_minutes']):

    if i == 0.0:
        merged_df.loc[index, 'delay_status'] = 0
    elif i<45.0:
        merged_df.loc[index, 'delay_status'] = 1
    elif i>=45.0 and i<=175.0:
        merged_df.loc[index, 'delay_status'] = 2
    else:
        merged_df.loc[index, 'delay_status'] = 3
        
from sklearn.model_selection import train_test_split

X = df_encoded[['type_departure', 'month_of_year_April', 'month_of_year_August',
       'month_of_year_December', 'month_of_year_February',
       'month_of_year_January', 'month_of_year_July', 'month_of_year_June',
       'month_of_year_March', 'month_of_year_May', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'status_cancelled','status_active','Temp Max','Temp Avg', 'Temp Min', 'Dew Point Max', 'Dew Point Avg',
       'Dew Point Min', 'Humidity Max', 'Humidity Avg', 'Humidity Min',
       'Wind Speed Max', 'Wind Speed Avg', 'Wind Speed Min', 'Pressure Max',
       'Pressure Avg', 'Pressure Min']]
y = merged_df['delay_status']
y=y.astype(int)

np.unique(merged_df['delay_status'])

array([0., 1., 2.])

# Model training by decision tree 

In [20]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_model.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=5, random_state=42)

## Predicting by decision tree model

In [21]:
y_valid_pred = tree_model.predict(X_test)
print("Validation Accuracy:", accuracy_score(y_test, y_valid_pred))
print("Classification Report on Validation Data:\n", classification_report(y_test, y_valid_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_valid_pred))

Validation Accuracy: 0.8790755262071812
Classification Report on Validation Data:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       275
           1       0.88      1.00      0.94      2130
           2       0.00      0.00      0.00        18

    accuracy                           0.88      2423
   macro avg       0.29      0.33      0.31      2423
weighted avg       0.77      0.88      0.82      2423

Confusion Matrix:
 [[   0  275    0]
 [   0 2130    0]
 [   0   18    0]]


C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model Training by Random Forest

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

multi_classifier = RandomForestClassifier(random_state=42)
multi_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## Predicting by random forest model

In [47]:
y_valid_pred = multi_classifier.predict(X_test)
print("Validation Accuracy:", accuracy_score(y_test, y_valid_pred))
print("Classification Report on Validation Data:\n", classification_report(y_test, y_valid_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_valid_pred))

Validation Accuracy: 0.8778373916632274
Classification Report on Validation Data:
               precision    recall  f1-score   support

           0       0.40      0.02      0.04       275
           1       0.88      1.00      0.93      2130
           2       0.00      0.00      0.00        18

    accuracy                           0.88      2423
   macro avg       0.43      0.34      0.33      2423
weighted avg       0.82      0.88      0.83      2423

Confusion Matrix:
 [[   6  269    0]
 [   9 2121    0]
 [   0   18    0]]


C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model training by GridSearchCV

In [58]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_neighbors': range(3, 15)  
}


grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


print("Best number of neighbors:", grid_search.best_params_)
print("Best accuracy score:", grid_search.best_score_)


Best number of neighbors: {'n_neighbors': 11}
Best accuracy score: 0.8741102597223576


## Predicting by GridSearchCV Model

In [59]:
y_valid_pred = grid_search.predict(X_test)
print("Validation Accuracy:", accuracy_score(y_test, y_valid_pred))
print("Classification Report on Validation Data:\n", classification_report(y_test, y_valid_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_valid_pred))

Validation Accuracy: 0.8728848534874123
Classification Report on Validation Data:
               precision    recall  f1-score   support

           0       0.33      0.05      0.09       275
           1       0.88      0.99      0.93      2130
           2       0.00      0.00      0.00        18

    accuracy                           0.87      2423
   macro avg       0.40      0.35      0.34      2423
weighted avg       0.81      0.87      0.83      2423

Confusion Matrix:
 [[  15  260    0]
 [  30 2100    0]
 [   1   17    0]]


C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ana\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Reading test data

In [60]:
test_df=pd.read_csv('Merge_data_test.csv')

## Hot Encoding Test data

In [61]:

df_encoded_test = pd.get_dummies(test_df, columns=['status','type','month_of_year','day_of_week','hour_of_day','airline.name',
       'airline.iataCode', 'airline.icaoCode', 'flight.number',
       'flight.iataNumber', 'flight.icaoNumber'])
df_encoded_test.columns

Index(['departure.iataCode', 'departure.icaoCode', 'departure.scheduledTime',
       'departure.estimatedRunway', 'departure.actualRunway',
       'arrival.iataCode', 'arrival.icaoCode', 'arrival.scheduledTime',
       'arrival.estimatedTime', 'departure.terminal',
       ...
       'flight.icaoNumber_uae613', 'flight.icaoNumber_uae615',
       'flight.icaoNumber_uae623', 'flight.icaoNumber_uae625',
       'flight.icaoNumber_uae8619', 'flight.icaoNumber_uae8621',
       'flight.icaoNumber_voz6098', 'flight.icaoNumber_voz6173',
       'flight.icaoNumber_voz6174', 'flight.icaoNumber_vsv850'],
      dtype='object', length=1972)

In [62]:

test_df['departure.actualRunway'] = pd.to_datetime(test_df['departure.actualRunway'])
test_df['departure.scheduledTime'] = pd.to_datetime(test_df['departure.scheduledTime'])


test_df['delay'] = test_df['departure.actualRunway'] - test_df['departure.scheduledTime']

test_df['delay_in_minutes'] = test_df['delay'].dt.total_seconds() / 60

test_df[['departure.scheduledTime', 'departure.actualRunway', 'delay_in_minutes']]


,departure.scheduledTime,departure.actualRunway,delay_in_minutes
0,2023-07-17 20:35:00,2023-07-17 20:46:00,11.0
1,2023-07-27 08:00:00,NaT,NaN
2,2023-07-27 08:00:00,NaT,NaN
3,2023-07-28 16:45:00,NaT,NaN
4,2023-07-19 04:15:00,2023-07-19 04:18:00,3.0
...,...,...,...
14905,2023-11-23 23:15:00,2023-11-23 23:27:00,12.0
14906,2023-11-25 18:45:00,2023-11-25 19:05:00,20.0
14907,2023-11-25 02:55:00,2023-11-25 03:07:00,12.0
14908,2023-11-27 08:35:00,NaT,NaN


## Predicting test data using decision tree

In [25]:

test_X = df_encoded_test[['type_departure', 'month_of_year_April', 'month_of_year_August',
       'month_of_year_December', 'month_of_year_February',
       'month_of_year_January', 'month_of_year_July', 'month_of_year_June',
       'month_of_year_March', 'month_of_year_May', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'status_cancelled','status_active','Temp Max','Temp Avg', 'Temp Min', 'Dew Point Max', 'Dew Point Avg',
       'Dew Point Min', 'Humidity Max', 'Humidity Avg', 'Humidity Min',
       'Wind Speed Max', 'Wind Speed Avg', 'Wind Speed Min', 'Pressure Max',
       'Pressure Avg', 'Pressure Min']]

# sahal=selector.transform(test_X)
# sahal=pca.transform(test_X)


test_pred = tree_model.predict(test_X)


In [26]:
np.unique(test_pred)

array([1])

In [27]:
label_mapping = {
    0: "No Delay",
    1: "Short Delay",
    2: "Moderate Delay",
    3: "Long Delay"
}


test_pred_mapped = [label_mapping[pred] for pred in test_pred]

In [28]:
np.unique((test_pred_mapped))

array(['Short Delay'], dtype='<U11')

In [29]:
submission_df = pd.DataFrame({
    'ID': range(1, len(test_pred_mapped) + 1),  
    'Delay': test_pred_mapped  
})
sahal=submission_df[:12914]

In [30]:
sahal.to_csv("submission_multi_decisiontree.csv", index=False)
print("Submission file created: submission_multi_decisiontree.csv")

Submission file created: submission_multi_decisiontree.csv


In [31]:
(sahal['Delay'].value_counts())

Short Delay    12914
Name: Delay, dtype: int64

## Predicting test data using Random forest

In [51]:

test_X = df_encoded_test[['type_departure', 'month_of_year_April', 'month_of_year_August',
       'month_of_year_December', 'month_of_year_February',
       'month_of_year_January', 'month_of_year_July', 'month_of_year_June',
       'month_of_year_March', 'month_of_year_May', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'status_cancelled','status_active','Temp Max','Temp Avg', 'Temp Min', 'Dew Point Max', 'Dew Point Avg',
       'Dew Point Min', 'Humidity Max', 'Humidity Avg', 'Humidity Min',
       'Wind Speed Max', 'Wind Speed Avg', 'Wind Speed Min', 'Pressure Max',
       'Pressure Avg', 'Pressure Min']]

# sahal=selector.transform(test_X)
# sahal=pca.transform(test_X)


test_pred = multi_classifier.predict(test_X)


In [52]:
np.unique(test_pred)

array([1])

In [53]:
label_mapping = {
    0: "No Delay",
    1: "Short Delay",
    2: "Moderate Delay",
    3: "Long Delay"
}


test_pred_mapped = [label_mapping[pred] for pred in test_pred]

In [54]:
np.unique((test_pred_mapped))

array(['Short Delay'], dtype='<U11')

In [55]:
submission_df = pd.DataFrame({
    'ID': range(1, len(test_pred_mapped) + 1),  
    'Delay': test_pred_mapped  
})
sahal=submission_df[:12914]

In [56]:
sahal.to_csv("submission_multi_randomforest.csv", index=False)
print("Submission file created: submission_multi_randomforest.csv")

Submission file created: submission_multi_randomforest.csv


In [57]:
(sahal['Delay'].value_counts())

Short Delay    12914
Name: Delay, dtype: int64

## Prediciting test Data by GridSearchCV

In [63]:

test_X = df_encoded_test[['type_departure', 'month_of_year_April', 'month_of_year_August',
       'month_of_year_December', 'month_of_year_February',
       'month_of_year_January', 'month_of_year_July', 'month_of_year_June',
       'month_of_year_March', 'month_of_year_May', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'status_cancelled','status_active','Temp Max','Temp Avg', 'Temp Min', 'Dew Point Max', 'Dew Point Avg',
       'Dew Point Min', 'Humidity Max', 'Humidity Avg', 'Humidity Min',
       'Wind Speed Max', 'Wind Speed Avg', 'Wind Speed Min', 'Pressure Max',
       'Pressure Avg', 'Pressure Min']]

# sahal=selector.transform(test_X)
# sahal=pca.transform(test_X)


test_pred = grid_search.predict(test_X)


In [64]:
np.unique(test_pred)

array([0, 1])

In [65]:
label_mapping = {
    0: "No Delay",
    1: "Short Delay",
    2: "Moderate Delay",
    3: "Long Delay"
}


test_pred_mapped = [label_mapping[pred] for pred in test_pred]

In [66]:
np.unique((test_pred_mapped))

array(['No Delay', 'Short Delay'], dtype='<U11')

In [67]:
submission_df = pd.DataFrame({
    'ID': range(1, len(test_pred_mapped) + 1),  
    'Delay': test_pred_mapped  
})
sahal=submission_df[:12914]

In [68]:
sahal.to_csv("submission_multi_gridsearch.csv", index=False)
print("Submission file created: submission_multi_gridsearch.csv")

Submission file created: submission_multi_gridsearch.csv


In [69]:
(sahal['Delay'].value_counts())

Short Delay    12789
No Delay         125
Name: Delay, dtype: int64